## Functions to clean dataframes
I created a function to clean the "df_pin" DataFrame, it can encapsulate the cleaning steps into a single function. 

Also I created an error handling to the function, it can use a try-except block to catch any potential errors that may occur during the cleaning process. Here's the function with error handling:

## Task 1: Clean the DataFrame that contains information about Pinterest posts

#### To clean the df_pin DataFrame you should perform the following transformations:

- Replace empty entries and entries with no relevant data in each column with Nones
- Perform the necessary transformations on the follower_count to ensure every entry is a number. Make sure the data type of this column is an int.
- Ensure that each column containing numeric data has a numeric data type
- Clean the data in the save_location column to include only the save location path
- Rename the index column to ind.
- Reorder the DataFrame columns to have the following column order:
  - ind
  - unique_id
  - title
  - description
  - follower_count
  - poster_name
  - tag_list
  - is_image_or_video
  - image_src
  - save_location
  - category

In [ ]:
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import IntegerType, LongType

def clean_pin_dataframe(df_pin):
    """
    Clean the DataFrame that contains information about Pinterest posts.

    Parameters:
    df_pin (DataFrame): DataFrame containing Pinterest posts data.

    Returns:
    df_pin_cleaned (DataFrame): Cleaned DataFrame.
    """
    try:
        # Replace empty entries and entries with no relevant data in each column with None
        df_pin_cleaned = df_pin.fillna("None")

        # Replace "k" with "000" and cast to integer for follower_count column
        df_pin_cleaned = df_pin_cleaned.withColumn("follower_count", 
                                                   regexp_replace(col("follower_count"), "k", "000").cast(IntegerType()))

        # Cast downloaded column to IntegerType
        df_pin_cleaned = df_pin_cleaned.withColumn("downloaded", df_pin_cleaned["downloaded"].cast(IntegerType()))

        # Cast index column to LongType
        df_pin_cleaned = df_pin_cleaned.withColumn("index", df_pin_cleaned["index"].cast(LongType()))

        # Rename index column to ind
        df_pin_cleaned = df_pin_cleaned.withColumnRenamed("index", "ind")

        # Reorder the DataFrame columns
        df_pin_cleaned = df_pin_cleaned.select("ind", "unique_id", "title", "description", "follower_count", 
                                               "poster_name", "tag_list", "is_image_or_video", "image_src", 
                                               "save_location", "category")

        return df_pin_cleaned

    except Exception as e:
        print(f"An error occurred during data cleaning: {str(e)}")
        return None


I can then call this function passing the "df_pin" DataFrame as an argument to clean it:

In [ ]:
cleaned_df_pin = clean_pin_dataframe(df_pin)
cleaned_df_pin.show()

+-----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
 ind| unique_id| title| description|follower_count| poster_name| tag_list|is_image_or_video| image_src| save_location| category|
+-----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
10138|927c4658-cc3f-4b9...|14 Amazing Things...|This Costa Rica i...| 10000|Wanderlust Chloe ...|Costa Rica Travel...| image|https://i.pinimg....| /data/travel| travel|
 3156|fa6e31a4-18c2-4ec...|Handprint Reindee...|This post may con...| 892000|Michelle {CraftyM...|Christmas Gifts F...| image|https://i.pinimg....|/data/diy-and-crafts|diy-and-crafts|
 5395|71f72304-3708-45f...|So startest du mi...|Der #Vermögensauf...| 3000|DIVDepot – Geld s...|Money Saving Chal...| image|https://i.pinimg....| /data/finance| finance|
 5494|8fb2af68-543b-463...|Dave Ramsey's 7 B...|If you love budge...| 26000|Living Low Key | ...|Financial Peace,F...| image|https://i.pinimg....| /data/finance| finance|
 2418|da8745a6-5160-46c...|100 DIY Christmas...|Here are the best...| 500000| HikenDip|Farmhouse Christm...| image|https://i.pinimg....| /data/christmas| christmas|
 1611|051e231c-7509-437...|Over The Years, P...|[PaidLink] It's m...| 22| Hair Accessories|Natural Beauty Ti...| image|https://i.pinimg....| /data/beauty| beauty|
 3813|49f7dfe7-2e5a-403...|Fun Interactive D...|Tired of Google C...| 9000|Science of Curiosity|Google Classroom,...| image|https://i.pinimg....| /data/education| education|
 7922|a584581c-1b38-473...|45 Top Life Quote...|summcoco gives yo...| 306000|Sumcoco | Decor I...|Life Quotes Love,...| image|https://i.pinimg....| /data/quotes| quotes|
 1643|41be1bcd-1ead-476...|White and Gold Ch...|Connecticut life ...| 84000| Lauren McBride|Christmas Dining ...| image|https://i.pinimg....| /data/christmas| christmas|
 5091|43459535-64e9-48a...|Buying Your First...|Buying your first...| 223|Edwin | Cash The ...|Paying Off Car Lo...| image|https://i.pinimg....| /data/finance| finance|
 9979|2b2abc85-fc51-481...|Paris in the Summ...|Are you traveling...| 3000| Petite in Paris|Torre Eiffel Pari...| image|https://i.pinimg....| /data/travel| travel|
 3032|208b07bc-e042-4e8...| Puffy Ghost Craft|This ghost craft ...| 56000| Thriving Home|Halloween Arts An...| video|https://i.pinimg....|/data/diy-and-crafts|diy-and-crafts|
 6809|f8ca698f-abfc-4b6...|It's Time to Ditc...|It's safe to assu...| 383000| Suburban Men|Business Casual D...| image|https://i.pinimg....| /data/mens-fashion| mens-fashion|
 3644|cde2b4e2-7711-4a8...|More Amazon Must ...|Amazon is a teach...| 41000|Teach Create Moti...|Classroom Hacks,5...| image|https://i.pinimg....| /data/education| education|
 6575|15c45907-657e-439...|Affordable Farmho...|Believe it or not...| 31000| Maria Louise Design|Farmhouse Remodel...| image|https://i.pinimg....| /data/home-decor| home-decor|
 6145|82e13a07-db99-43a...|HOLIDAY MANTLE DE...|Holiday mantle de...| 83000| Stylin by Aylin|Winter Home Decor...| image|https://i.pinimg....| /data/home-decor| home-decor|
 6145|82e13a07-db99-43a...|HOLIDAY MANTLE DE...|Holiday mantle de...| 83000| Stylin by Aylin|Winter Home Decor...| image|https://i.pinimg....| /data/home-decor| home-decor|
 4|55abcd28-bda1-445...|Mexican Artist Us...|Mexican artist En...| null| Bored Panda|Girl Drawing Sket...| image|https://i.pinimg....| /data/art| art|
 4315|21b59ba9-829d-4c3...|Podcasts for Teac...|Podcasts for Teac...| 25000| Math Giraffe|Middle School Cla...| image|https://i.pinimg....| /data/education| education|
 5069|b75b6f87-deb3-444...|The Vault: Curate...|Sacramento Califo...| null| Style Me Pretty|60th Anniversary ...| image|https://i.pinimg....|/data/event-planning|event-planning|
+-----+--------------------+--------------------+--

## Functions to clean dataframes
I created a function to clean the "df_geo" DataFrame, it can encapsulate the cleaning steps into a single function. 

Also I created an error handling to the function, it can use a try-except block to catch any potential errors that may occur during the cleaning process. Here's the function with error handling:

## Task 2:

#### To clean the df_geo DataFrame you should perform the following transformations:

- Create a new column coordinates that contains an array based on the latitude and longitude columns
- Drop the latitude and longitude columns from the DataFrame
- Convert the timestamp column from a string to a timestamp data type
- Reorder the DataFrame columns to have the following column order:
  - ind
  - country
  - coordinates
  - timestamp

In [ ]:
from pyspark.sql.functions import array, col, to_timestamp

def clean_geo_dataframe(df_geo):
    """
    Clean the DataFrame that contains information about Pinterest geolocation data.

    Parameters:
    df_geo (DataFrame): DataFrame containing Pinterest geolocation data.

    Returns:
    df_geo_cleaned (DataFrame): Cleaned DataFrame.
    """
    try:
        # Create a new column coordinates containing an array of latitude and longitude
        df_geo_cleaned = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))

        # Drop the latitude and longitude columns
        df_geo_cleaned = df_geo_cleaned.drop("latitude", "longitude")

        # Convert the timestamp column to a timestamp data type
        df_geo_cleaned = df_geo_cleaned.withColumn("timestamp", to_timestamp("timestamp"))

        # Reorder the DataFrame columns
        df_geo_cleaned = df_geo_cleaned.select("ind", "country", "coordinates", "timestamp")

        return df_geo_cleaned

    except Exception as e:
        print(f"An error occurred during data cleaning: {str(e)}")
        return None


I can then call this function passing the "df_geo" DataFrame as an argument to clean it:

In [ ]:
cleaned_df_geo = clean_pin_dataframe(df_geo)
cleaned_df_geo.show()


+-----+--------------------+--------------------+-------------------+
 ind| country| coordinates| timestamp|
+-----+--------------------+--------------------+-------------------+
 2418|Antarctica (the t...|[-88.4642, -171.061]|2022-05-27 11:30:59|
 9270|Bouvet Island (Bo...|[-84.3984, -144.933]|2022-05-23 03:24:24|
10794|Cocos (Keeling) I...|[-89.5236, -154.567]|2022-01-01 02:26:50|
 5395|British Virgin Is...|[-6.93422, -24.9989]|2020-08-15 19:30:19|
 2074|Central African R...| [-52.3213, -50.11]|2019-11-03 05:41:59|
 1170|Libyan Arab Jamah...| [56.1198, 20.2963]|2019-08-07 22:17:07|
 9550|British Virgin Is...| [-84.918, 7.23235]|2022-03-23 00:25:45|
 7922| Antigua and Barbuda|[-88.0974, -172.052]|2021-01-27 09:14:19|
 9979| Dominican Republic| [14.9967, -120.682]|2018-07-18 19:01:46|
 6575| Dominican Republic| [81.7192, 61.5152]|2020-12-29 22:59:37|
 3032| American Samoa|[-66.7253, -122.489]|2018-04-14 09:28:18|
 637| American Samoa|[-41.2244, -158.022]|2018-06-06 01:25:58|
10321| Burkina Faso|[-89.1005, -151.255]|2022-10-09 08:06:50|
 2923| Cote d'Ivoire|[-84.6302, -164.507]|2019-09-08 22:53:09|
 8304| French Guiana| [-28.8852, -164.87]|2019-09-13 04:50:29|
 4315| Cote d'Ivoire| [-45.8508, 66.1003]|2019-12-15 03:51:28|
 4249| Afghanistan|[-88.5478, -174.971]|2021-09-01 11:10:02|
 6809| Bangladesh|[-89.4254, -161.818]|2020-04-17 13:20:43|
 5069| Azerbaijan|[-63.0063, -157.474]|2021-03-20 09:32:44|
 6145| Mozambique|[-65.9079, -143.845]|2019-12-05 02:09:44|
+-----+--------------------+--------------------+-------------------+
only showing top 20 rows

## Task 3:
1. Create a new column user_name by concatenating first_name and last_name
2. Drop the first_name and last_name columns
3. Convert the date_joined column to a timestamp data type
4. Reorder the DataFrame columns
   - "ind", "user_name", "age", "date_joined"

In [ ]:
from pyspark.sql.functions import concat, col, to_timestamp

def clean_user_dataframe(df_user):
    """
    Clean the DataFrame that contains information about Pinterest user data.

    Parameters:
    df_user (DataFrame): DataFrame containing Pinterest user data.

    Returns:
    df_user_cleaned (DataFrame): Cleaned DataFrame.
    """
    try:
        # 1. Create a new column user_name by concatenating first_name and last_name
        df_user_cleaned = df_user.withColumn("user_name", concat(col("first_name"), col("last_name")))

        # 2. Drop the first_name and last_name columns
        df_user_cleaned = df_user_cleaned.drop("first_name", "last_name")

        # 3. Convert the date_joined column to a timestamp data type
        df_user_cleaned = df_user_cleaned.withColumn("date_joined", to_timestamp("date_joined"))

        # 4. Reorder the DataFrame columns
        df_user_cleaned = df_user_cleaned.select("ind", "user_name", "age", "date_joined")

        return df_user_cleaned

    except Exception as e:
        print(f"An error occurred during data cleaning: {str(e)}")
        return None

I can then call this function passing the "df_user" DataFrame as an argument to clean it:

In [ ]:
cleaned_df_user = clean_pin_dataframe(df_user)
cleaned_df_user.show()

+-----+------------------+---+-------------------+
 ind| user_name|age| date_joined|
+-----+------------------+---+-------------------+
 4249|AlexandriaAlvarado| 20|2015-10-23 04:13:23|
 8731| AndreaAlexander| 21|2015-11-10 09:27:42|
 8731| AndreaAlexander| 21|2015-11-10 09:27:42|
 8731| AndreaAlexander| 21|2015-11-10 09:27:42|
10137| DanielMatthews| 34|2016-01-23 03:59:37|
 8731| AndreaAlexander| 21|2015-11-10 09:27:42|
 8887| AustinRodriguez| 24|2016-03-31 20:56:39|
 4315| MichellePrince| 36|2015-12-20 16:38:13|
10625| ChristianLang| 32|2017-10-10 20:09:33|
 4315| MichellePrince| 36|2015-12-20 16:38:13|
 1514| MichaelPerkins| 40|2017-06-28 16:17:39|
 4315| MichellePrince| 36|2015-12-20 16:38:13|
10625| ChristianLang| 32|2017-10-10 20:09:33|
10625| ChristianLang| 32|2017-10-10 20:09:33|
 9672| JenniferHudson| 22|2016-02-11 20:46:04|
 1313| BrittanyJones| 32|2016-04-02 03:51:23|
 1313| BrittanyJones| 32|2016-04-02 03:51:23|
10794| ThomasTurner| 34|2016-12-22 00:02:02|
 1313| BrittanyJones| 32|2016-04-02 03:51:23|
10794| ThomasTurner| 34|2016-12-22 00:02:02|
+-----+------------------+---+-------------------+
only showing top 20 rows

## Task 4: Find the most popular category by country

#### Find the most popular Pinterest category people post to based on their country.
Your query should return a DataFrame that contains the following columns:
- country
- category
- category_count, a new column containing the desired query output

### Start

In [ ]:
from pyspark.sql.functions import col, max
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")

# Join df_pin with df_geo on the index column
df_combined = df_pin_alias.join(df_geo_alias, df_pin_alias.ind == df_geo_alias.ind)

# Group by country and category, then count the occurrences of each category for each country
df_category_count = df_combined.groupBy(df_geo_alias["geo_country"], df_pin_alias["category"]).count()

# Create a window partitioned by country, ordered by the count descending
windowSpec = Window.partitionBy("geo_country").orderBy(col("count").desc())

# Use row_number to assign a rank to each row within each country partition
df_ranked_categories = df_category_count.withColumn("rank", row_number().over(windowSpec))

# Filter to get only the top category (rank = 1) for each country
df_top_category_per_country = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Select the desired columns
# This is the erroneours line.
####### df_most_popular = df_most_popular.select("geo_country", "category", "count")

# Filter to get only the top category (rank = 1) for each country
df_most_popular = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Rename the columns
df_most_popular = df_most_popular.withColumnRenamed("geo_country", "country")
df_most_popular = df_most_popular.withColumnRenamed("count", "category_count")

# Display the result
df_most_popular.show()

+--------------------+--------------+--------------+
 country| category|category_count|
+--------------------+--------------+--------------+
 Afghanistan| education| 1|
 Albania| mens-fashion| 4|
 Algeria| quotes| 1|
 American Samoa|diy-and-crafts| 1|
 Angola|diy-and-crafts| 4|
 Anguilla| home-decor| 1|
Antarctica (the t...| christmas| 1|
 Antigua and Barbuda| quotes| 1|
 Armenia|diy-and-crafts| 5|
 Aruba| travel| 1|
 Australia| mens-fashion| 1|
 Austria|event-planning| 1|
 Azerbaijan|event-planning| 1|
 Bahamas| art| 1|
 Bangladesh| mens-fashion| 1|
 Barbados| travel| 4|
 Botswana| tattoos| 1|
Bouvet Island (Bo...| tattoos| 1|
British Virgin Is...| travel| 1|
 Bulgaria| finance| 1|
+--------------------+--------------+--------------+
only showing top 20 rows

In [ ]:
from pyspark.sql.functions import col, max
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")

# Join df_pin with df_geo on the index column
df_combined = df_pin_alias.join(df_geo_alias, df_pin_alias.ind == df_geo_alias.ind)

# Group by country and category, then count the occurrences of each category for each country
df_category_count = df_combined.groupBy(df_geo_alias["geo_country"], df_pin_alias["category"]).count()

# Create a window partitioned by country, ordered by the count descending
windowSpec = Window.partitionBy("geo_country").orderBy(col("count").desc())

# Use row_number to assign a rank to each row within each country partition
df_ranked_categories = df_category_count.withColumn("rank", row_number().over(windowSpec))

# Filter to get only the top category (rank = 1) for each country
df_top_category_per_country = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Select the desired columns
# This is the erroneours line.
####### df_most_popular = df_most_popular.select("geo_country", "category", "count")

# Filter to get only the top category (rank = 1) for each country
df_most_popular = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Rename the columns
df_most_popular = df_most_popular.withColumnRenamed("geo_country", "country")
df_most_popular = df_most_popular.withColumnRenamed("count", "category_count")

# Display the result
df_most_popular.show()

+--------------------+--------------+--------------+
 country| category|category_count|
+--------------------+--------------+--------------+
 Afghanistan| education| 1|
 Albania| mens-fashion| 4|
 Algeria| quotes| 1|
 American Samoa| art| 1|
 Angola|diy-and-crafts| 4|
 Anguilla| home-decor| 1|
Antarctica (the t...| christmas| 1|
 Antigua and Barbuda| quotes| 1|
 Armenia|diy-and-crafts| 5|
 Aruba| travel| 1|
 Australia| mens-fashion| 1|
 Austria| travel| 1|
 Azerbaijan|event-planning| 1|
 Bahamas| art| 1|
 Bangladesh| art| 1|
 Barbados| travel| 4|
 Botswana| tattoos| 1|
Bouvet Island (Bo...| tattoos| 1|
British Virgin Is...| travel| 1|
 Bulgaria| finance| 1|
+--------------------+--------------+--------------+
only showing top 20 rows

### End

In [ ]:
from pyspark.sql.functions import col

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")

# Join df_pin with df_geo on the index column
df_combined = df_pin_alias.join(df_geo_alias, df_pin_alias.ind == df_geo_alias.ind)
print(df_combined)
# Group by country and category, then count the occurrences of each category for each country
df_category_count = df_combined.groupBy("geo_country", "pin.category").count()
print(df_category_count)
# Find the maximum count for each country
max_counts = df_category_count.groupBy("geo_country").max("count")
print(max_counts)

max_counts.show()


DataFrame[ind: bigint, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string, ind: bigint, geo_country: string, coordinates: array<double>, timestamp: timestamp]
DataFrame[geo_country: string, category: string, count: bigint]
DataFrame[geo_country: string, max(count): bigint]
+--------------------+----------+
 geo_country|max(count)|
+--------------------+----------+
 Anguilla| 1|
 Fiji| 1|
 Cambodia| 1|
 Afghanistan| 1|
 Maldives| 1|
 Sudan| 1|
British Virgin Is...| 1|
 Algeria| 1|
 Angola| 4|
 Albania| 4|
 India| 1|
 Bahamas| 1|
 American Samoa| 1|
 Burundi| 1|
Central African R...| 1|
 Bangladesh| 1|
 Barbados| 4|
 Congo| 1|
Cocos (Keeling) I...| 1|
 Cote d'Ivoire| 1|
+--------------------+----------+
only showing top 20 rows

In [ ]:
from pyspark.sql.functions import col

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")

# Join df_pin with df_geo on the index column
df_combined = df_pin_alias.join(df_geo_alias, df_pin_alias.ind == df_geo_alias.ind)
print(df_combined)
# Group by country and category, then count the occurrences of each category for each country
df_category_count = df_combined.groupBy("geo_country", "pin.category").count()
print(df_category_count)
# Find the maximum count for each country
max_counts = df_category_count.groupBy("geo_country").max("count")
print(max_counts)

max_counts.show()

DataFrame[ind: bigint, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string, ind: bigint, geo_country: string, coordinates: array<double>, timestamp: timestamp]
DataFrame[geo_country: string, category: string, count: bigint]
DataFrame[geo_country: string, max(count): bigint]
+--------------------+----------+
 geo_country|max(count)|
+--------------------+----------+
 Anguilla| 1|
 Fiji| 1|
 Afghanistan| 1|
 Cambodia| 1|
 Maldives| 1|
 Sudan| 1|
British Virgin Is...| 1|
 Algeria| 1|
 Angola| 4|
 Albania| 4|
 India| 1|
 Bahamas| 1|
 American Samoa| 1|
 Burundi| 1|
Central African R...| 1|
 Bangladesh| 1|
 Barbados| 4|
 Congo| 1|
Cocos (Keeling) I...| 1|
 Cote d'Ivoire| 1|
+--------------------+----------+
only showing top 20 rows

In [ ]:
from pyspark.sql.functions import col

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")
print(df_pin_alias)
print(df_geo_alias)

DataFrame[ind: bigint, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string]
DataFrame[ind: bigint, geo_country: string, coordinates: array<double>, timestamp: timestamp]

## Task 5: Find which was the most popular category each year

Task 5: Find which was the most popular category each year

In [ ]:
from pyspark.sql.functions import col, year
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Combine the data from df_pin and df_geo
df_combined = df_pin.join(df_geo, df_pin.ind == df_geo.ind)

# Extract the year from the timestamp column
df_combined = df_combined.withColumn("year", year("timestamp"))

# Group by year and category, then count the occurrences of each category for each year
df_category_count = df_combined.groupBy("year", "category").count()

# Create a window partitioned by year, ordered by the count descending
windowSpec = Window.partitionBy("year").orderBy(col("count").desc())

# Use row_number to assign a rank to each row within each year partition
df_ranked_categories = df_category_count.withColumn("rank", row_number().over(windowSpec))

# Filter to get only the top category (rank = 1) for each year
df_top_category_per_year = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Rename the columns
#df_top_category_per_year = df_top_category_per_year.withColumnRenamed("category", "most_popular_category")
df_top_category_per_year = df_top_category_per_year.withColumnRenamed("count", "category_count")

# Display the result
df_top_category_per_year.show()


+----+--------------+--------------+
year| category|category_count|
+----+--------------+--------------+
2018| vehicles| 4|
2019|diy-and-crafts| 5|
2020| mens-fashion| 6|
2021| quotes| 2|
2022| christmas| 2|
+----+--------------+--------------+

In [ ]:
from pyspark.sql.functions import col, year
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Combine the data from df_pin and df_geo
df_combined = df_pin.join(df_geo, df_pin.ind == df_geo.ind)

# Extract the year from the timestamp column and rename it to post_year
df_combined = df_combined.withColumn("post_year", year("timestamp")).drop("year")

# Group by post_year and category, then count the occurrences of each category for each post_year
df_category_count = df_combined.groupBy("post_year", "category").count()

# Create a window partitioned by post_year, ordered by the count descending
windowSpec = Window.partitionBy("post_year").orderBy(col("count").desc())

# Use row_number to assign a rank to each row within each post_year partition
df_ranked_categories = df_category_count.withColumn("rank", row_number().over(windowSpec))

# Filter to get only the top category (rank = 1) for each post_year
df_top_category_per_year = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Rename the columns
df_top_category_per_year = df_top_category_per_year.withColumnRenamed("count", "category_count")

# Display the result
df_top_category_per_year.show()


+---------+--------------+--------------+
post_year| category|category_count|
+---------+--------------+--------------+
 2018| vehicles| 4|
 2019|diy-and-crafts| 5|
 2020| mens-fashion| 6|
 2021| tattoos| 2|
 2022| christmas| 2|
+---------+--------------+--------------+

## Task 6: Find the user with most followers in each country

### Step 1: For each country find the user with the most followers.
Your query should return a DataFrame that contains the following columns:

- country
- poster_name
- follower_count

In [ ]:
from pyspark.sql.functions import col, max
from pyspark.sql.window import Window

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")

# Join df_pin with df_geo on the index column
df_combined = df_pin_alias.join(df_geo_alias, df_pin_alias.ind == df_geo_alias.ind)

# Group by country and user (poster_name), then find the maximum follower count for each group
df_max_followers_per_user = df_combined.groupBy("geo_country", "poster_name").agg(max("follower_count").alias("follower_count"))

# Create a window partitioned by country, ordered by the follower_count descending
windowSpec = Window.partitionBy("geo_country").orderBy(col("follower_count").desc())

# Use row_number to assign a rank to each row within each country partition
df_ranked_users = df_max_followers_per_user.withColumn("rank", row_number().over(windowSpec))

# Filter to keep only the top user (rank = 1) for each country
df_top_users_per_country = df_ranked_users.filter(col("rank") == 1).drop("rank")

# Rename the columns
df_top_users_per_country = df_top_users_per_country.withColumnRenamed("geo_country", "country")

# Select the desired columns
df_top_users_per_country = df_top_users_per_country.select("country", "poster_name", "follower_count")

# Display the result
df_top_users_per_country.show()



+--------------------+--------------------+--------------+
 country| poster_name|follower_count|
+--------------------+--------------------+--------------+
 Afghanistan| Walmart| null|
 Albania| Bored Panda| null|
 Algeria| YourTango| 942000|
 American Samoa| Thriving Home| 56000|
 Angola| CraftGossip.com| 502000|
 Anguilla|Kristen | Lifesty...| 92000|
Antarctica (the t...| HikenDip| 500000|
 Antigua and Barbuda|Sumcoco | Decor I...| 306000|
 Armenia|Michelle {CraftyM...| 892000|
 Aruba| TheTrendSpotter| 211000|
 Australia| Cultura Colectiva| null|
 Austria|Wanderlust Chloe ...| 10000|
 Azerbaijan| Style Me Pretty| null|
 Bahamas|The Kitchen Table...| 221000|
 Bangladesh| Suburban Men| 383000|
 Barbados| Nicki | 28000|
 Botswana| RapidLeaks| 4000|
Bouvet Island (Bo...| POPSUGAR| null|
British Virgin Is...|Visit USA Parks |...| 4000|
 Bulgaria|Living Low Key | ...| 26000|
+--------------------+--------------------+--------------+
only showing top 20 rows

## Step 2: Based on the above query, find the country with the user with most followers.
Your query should return a DataFrame that contains the following columns:
- country
- follower_count

This DataFrame should have only one entry.

In [ ]:
from pyspark.sql.functions import col, max

# Group by country and find the maximum follower count across all users in each country
df_max_followers_per_country = df_top_users_per_country.groupBy("country").agg(max("follower_count").alias("follower_count"))

# Create a window partitioned by country, ordered by the follower_count descending
windowSpec = Window.orderBy(col("follower_count").desc())

# Use row_number to assign a rank to each row
df_ranked_countries = df_max_followers_per_country.withColumn("rank", row_number().over(windowSpec))

# Filter to keep only the top country (rank = 1)
df_country_with_most_followers = df_ranked_countries.filter(col("rank") == 1).drop("rank")

# Display the result
df_country_with_most_followers.show()


+-------+--------------+
country|follower_count|
+-------+--------------+
Algeria| 942000|
+-------+--------------+

### Task 7: Find the most popular category for different age groups

What is the most popular category people post to based on the following age groups:
- 18-24
- 25-35
- 36-50
- +50

Your query should return a DataFrame that contains the following columns:
- age_group, a new column based on the original age column
- category
- category_count, a new column containing the desired query output

In [ ]:
from pyspark.sql.functions import when

# Define age groups based on the original age column
df_user_age_grouped = df_user.withColumn("age_group",
                                         when(col("age").between(18, 24), "18-24")
                                         .when(col("age").between(25, 35), "25-35")
                                         .when(col("age").between(36, 50), "36-50")
                                         .otherwise("50+"))

# Join the combined DataFrame with df_pin on the index column
df_combined = df_pin.join(df_user_age_grouped, df_pin.ind == df_user_age_grouped.ind)

# Group by age group and category, then count the occurrences of each category for each age group
df_category_count = df_combined.groupBy("age_group", "category").count()

# Create a window partitioned by age group, ordered by the count descending
windowSpec = Window.partitionBy("age_group").orderBy(col("count").desc())

# Use row_number to assign a rank to each row within each age group partition
df_ranked_categories = df_category_count.withColumn("rank", row_number().over(windowSpec))

# Filter to get only the top category (rank = 1) for each age group
df_most_popular_per_age_group = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Rename the columns
df_most_popular_per_age_group = df_most_popular_per_age_group.withColumnRenamed("count", "category_count")

# Display the result
df_most_popular_per_age_group.show()


+---------+--------------+--------------+
age_group| category|category_count|
+---------+--------------+--------------+
 18-24| mens-fashion| 17|
 25-35|diy-and-crafts| 12|
 36-50| finance| 5|
 50+| beauty| 1|
+---------+--------------+--------------+

### Task 8: Find the median follower count for different age groups

What is the median follower count for users in the following age groups:

- 18-24
- 25-35
- 36-50
- +50

Your query should return a DataFrame that contains the following columns:

- age_group, a new column based on the original age column
- median_follower_count, a new column containing the desired query output

In [ ]:
from pyspark.sql.functions import col, expr

# Define age groups based on the original age column
df_user_age_grouped = df_user.withColumn("age_group",
                                         when(col("age").between(18, 24), "18-24")
                                         .when(col("age").between(25, 35), "25-35")
                                         .when(col("age").between(36, 50), "36-50")
                                         .otherwise("50+"))

# Join the user DataFrame with the pin DataFrame on the index column
df_combined = df_pin.join(df_user_age_grouped, df_pin.ind == df_user_age_grouped.ind)

# Group by age group and calculate the median follower count for each age group
df_median_follower_count = df_combined.groupBy("age_group") \
    .agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

# Display the result
df_median_follower_count.show()


+---------+---------------------+
age_group|median_follower_count|
+---------+---------------------+
 18-24| 211000|
 25-35| 43000|
 36-50| 6000|
 50+| 908|
+---------+---------------------+

## Task 9: Find how many users have joined each year?
Find how many users have joined between 2015 and 2020.

Your query should return a DataFrame that contains the following columns:

- post_year, a new column that contains only the year from the timestamp column
- number_users_joined, a new column containing the desired query output

In [ ]:
from pyspark.sql.functions import year, countDistinct

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Group by the join year and count the distinct users
df_number_users_joined = df_filtered_users.groupBy("join_year").agg(countDistinct("user_name").alias("number_users_joined"))

# Display the result
df_number_users_joined.show()


+---------+-------------------+
join_year|number_users_joined|
+---------+-------------------+
 2015| 24|
 2016| 27|
 2017| 8|
+---------+-------------------+

## Task 10: Find the median follower count of users based on their joining year

Find the median follower count of users have joined between 2015 and 2020.

Your query should return a DataFrame that contains the following columns:

- post_year, a new column that contains only the year from the timestamp column
- median_follower_count, a new column containing the desired query output


In [ ]:
from pyspark.sql.functions import year
from pyspark.sql import Window
import pyspark.sql.functions as F

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate the median follower count for each year
windowSpec = Window.partitionBy("join_year")
df_median_follower_count = df_filtered_users.withColumn("median_follower_count", F.expr("percentile_approx(age, 0.5)").over(windowSpec)).distinct()

# Display the result
df_median_follower_count.show()


+-----+------------------+---+-------------------+---------+---------------------+
 ind| user_name|age| date_joined|join_year|median_follower_count|
+-----+------------------+---+-------------------+---------+---------------------+
 5730| RachelDavis| 36|2015-12-08 20:02:43| 2015| 22|
 2923| BrianNelson| 26|2015-11-11 03:20:57| 2015| 22|
 857| AndrewBurke| 20|2015-11-14 17:38:31| 2015| 22|
 7528| AbigailAli| 20|2015-10-24 11:23:51| 2015| 22|
 4249|AlexandriaAlvarado| 20|2015-10-23 04:13:23| 2015| 22|
 8731| AndreaAlexander| 21|2015-11-10 09:27:42| 2015| 22|
 6063| CoreyAndrews| 23|2015-11-25 13:36:22| 2015| 22|
 6145| EmilyHawkins| 30|2015-12-25 20:24:37| 2015| 22|
 8304| CharlesBerry| 25|2015-12-28 04:21:39| 2015| 22|
 3156| AndrewBaker| 22|2015-12-21 08:06:54| 2015| 22|
 428| ClaudiaAdams| 20|2015-11-28 02:20:29| 2015| 22|
10138| CarolSilva| 22|2015-12-31 14:57:02| 2015| 22|
 7922| DeniseAdams| 21|2015-11-12 06:21:36| 2015| 22|
 9875| BrendanJoseph| 26|2015-12-20 10:28:00| 2015| 22|
 4315| MichellePrince| 36|2015-12-20 16:38:13| 2015| 22|
 8237| AaronAbbott| 20|2015-10-23 16:08:41| 2015| 22|
 3089| AbigailAli| 20|2015-10-24 11:23:51| 2015| 22|
 5494| AnneAllen| 27|2015-12-16 15:20:05| 2015| 22|
10883| CarlHampton| 25|2015-11-26 03:43:03| 2015| 22|
10321| AndreaBurke| 20|2015-12-12 11:20:08| 2015| 22|
+-----+------------------+---+-------------------+---------+---------------------+
only showing top 20 rows

## Task 11: Find the median follower count of users based on their joining year and age group

Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.
Your query should return a DataFrame that contains the following columns:

- age_group, a new column based on the original age column
- post_year, a new column that contains only the year from the timestamp column
- median_follower_count, a new column containing the desired query output

In [ ]:
from pyspark.sql.functions import year, expr, col
from pyspark.sql import Window
import pyspark.sql.functions as F

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate age group
df_filtered_users = df_filtered_users.withColumn("age_group", F.when(col("age") < 20, "0-19")
                                                     .when((col("age") >= 20) & (col("age") < 30), "20-29")
                                                     .when((col("age") >= 30) & (col("age") < 40), "30-39")
                                                     .otherwise("40+"))

# Group by age group, join year, and calculate the median follower count
df_median_follower_count = df_filtered_users.groupBy("age_group", "join_year") \
    .agg(expr("percentile_approx(ind, 0.5)").alias("median_follower_count"))

# Add a column for post_year containing only the year from the timestamp column
df_median_follower_count = df_median_follower_count.withColumn("post_year", col("join_year"))

# Reorder columns
df_median_follower_count = df_median_follower_count.select("age_group", "post_year", "median_follower_count")

# Display the result
df_median_follower_count.show()


+---------+---------+---------------------+
age_group|post_year|median_follower_count|
+---------+---------+---------------------+
 20-29| 2015| 7528|
 40+| 2016| 3813|
 20-29| 2016| 5069|
 30-39| 2016| 2863|
 20-29| 2017| 5091|
 30-39| 2015| 5730|
 40+| 2017| 1611|
 30-39| 2017| 10625|
+---------+---------+---------------------+

In [ ]:
from pyspark.sql.functions import year, expr, col
from pyspark.sql import Window
import pyspark.sql.functions as F

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate age group
#df_filtered_users = df_filtered_users.withColumn("age_group", F.when(col("age") < 20, "0-19")
                                                     #.when((col("age") >= 20) & (col("age") < 30), #"20-29")
                                                     #.when((col("age") >= 30) & (col("age") < 40), #"30-39")
                                                     #.otherwise("40+"))

# Group by age group, join year, and calculate the median follower count
df_median_follower_count = df_filtered_users.groupBy("join_year") \
    .agg(expr("percentile_approx(ind, 0.5)").alias("median_follower_count"))

# Display the result
df_median_follower_count.show()

+---------+---------------------+
join_year|median_follower_count|
+---------+---------------------+
 2015| 6145|
 2016| 4949|
 2017| 5091|
+---------+---------------------+

In [ ]:
from pyspark.sql.functions import year, expr, col
from pyspark.sql import Window
import pyspark.sql.functions as F

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate age group
df_filtered_users = df_filtered_users.withColumn("age_group", F.when(col("age") < 20, "0-19")
                                                     .when((col("age") >= 20) & (col("age") < 30), "20-29")
                                                     .when((col("age") >= 30) & (col("age") < 40), "30-39")
                                                     .otherwise("40+"))

# Group by age group, join year, and calculate the median follower count
df_median_follower_count = df_filtered_users.groupBy("age_group", "join_year") \
    .agg(expr("percentile_approx(ind, 0.5)").alias("median_follower_count"))

# Display the result
df_median_follower_count.show()


+---------+---------+---------------------+
age_group|join_year|median_follower_count|
+---------+---------+---------------------+
 30-39| 2016| 2863|
 20-29| 2015| 7528|
 30-39| 2015| 5730|
 20-29| 2016| 5069|
 40+| 2017| 1611|
 20-29| 2017| 5091|
 40+| 2016| 3813|
 30-39| 2017| 10625|
+---------+---------+---------------------+

In [ ]:
from pyspark.sql.functions import expr
from pyspark.sql.window import Window

# Assuming follower information is contained within df_pin, you can filter the relevant columns
df_followers = df_pin.select("ind", "follower_count")

# Now you have a DataFrame df_followers containing the unique_id and follower_count columns


# Assuming df_followers is properly defined and contains the necessary columns
# Join df_filtered_users with df_followers on a common key (e.g., unique_id)
print(df_filtered_users)
print(df_followers)
print(df_followers.columns)  # This line is causing the error

# Correcting the error: Accessing the columns attribute of df_followers
print(df_followers.columns)


df_filtered_users_with_followers = df_filtered_users.join(df_followers, df_filtered_users["ind"] == df_followers["ind"], how="inner")
print(df_filtered_users)
print(df_followers)
# Once the join is successful, you can proceed with the rest of the code to calculate the median follower count

# Calculate the median follower count for each year and age group
windowSpec = Window.partitionBy("join_year", "age_group")
df_median_follower_count = df_filtered_users_with_followers.withColumn(
    "median_follower_count",
    expr("percentile_approx(follower_count, 0.5)").over(windowSpec)
).select("age_group", "join_year", "median_follower_count").distinct()

# Display the result
df_median_follower_count.show()


DataFrame[ind: bigint, user_name: string, age: bigint, date_joined: timestamp, join_year: int, age_group: string]
DataFrame[ind: bigint, follower_count: int]
['ind', 'follower_count']
['ind', 'follower_count']
DataFrame[ind: bigint, user_name: string, age: bigint, date_joined: timestamp, join_year: int, age_group: string]
DataFrame[ind: bigint, follower_count: int]
+---------+---------+---------------------+
age_group|join_year|median_follower_count|
+---------+---------+---------------------+
 18-24| 2015| 211000|
 25-35| 2015| 51000|
 36-50| 2015| 0|
 18-24| 2016| 502000|
 25-35| 2016| 43000|
 36-50| 2016| 17000|
 50+| 2016| 908|
 18-24| 2017| 940|
 25-35| 2017| 8000|
 36-50| 2017| 6000|
+---------+---------+---------------------+

In [ ]:
# Assuming df_followers is properly defined and contains the necessary columns
# Join df_filtered_users with df_followers on a common key (e.g., unique_id)
print(df_filtered_users)
print(df_followers)
print(df_followers.columns)  # This line is causing the error

# Correcting the error: Accessing the columns attribute of df_followers
print(df_followers.columns)

df_filtered_users_with_followers = df_filtered_users.join(df_followers, df_filtered_users["ind"] == df_followers["ind"], how="inner")
print(df_filtered_users)
print(df_followers)
# Once the join is successful, you can proceed with the rest of the code to calculate the median follower count

# Calculate the median follower count for each year and age group
windowSpec = Window.partitionBy("join_year", "age_group")
df_median_follower_count = df_filtered_users_with_followers.withColumn(
    "median_follower_count",
    expr("percentile_approx(follower_count, 0.5)").over(windowSpec)
).select("age_group", "join_year", "median_follower_count").distinct()

# Display the result
df_median_follower_count.show()


DataFrame[ind: bigint, user_name: string, age: bigint, date_joined: timestamp, join_year: int, age_group: string]
DataFrame[ind: bigint, follower_count: int]
['ind', 'follower_count']
['ind', 'follower_count']
DataFrame[ind: bigint, user_name: string, age: bigint, date_joined: timestamp, join_year: int, age_group: string]
DataFrame[ind: bigint, follower_count: int]
+---------+---------+---------------------+
age_group|join_year|median_follower_count|
+---------+---------+---------------------+
 18-24| 2015| 211000|
 25-35| 2015| 51000|
 36-50| 2015| 0|
 18-24| 2016| 502000|
 25-35| 2016| 43000|
 36-50| 2016| 17000|
 50+| 2016| 908|
 18-24| 2017| 940|
 25-35| 2017| 8000|
 36-50| 2017| 6000|
+---------+---------+---------------------+

In [ ]:
print(df_pin)

DataFrame[ind: bigint, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string]

In [ ]:
from pyspark.sql.functions import year
from pyspark.sql import Window
import pyspark.sql.functions as F

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate the median follower count for each year
windowSpec = Window.partitionBy("join_year")
df_median_follower_count = df_filtered_users.withColumn("median_follower_count", F.expr("percentile_approx(age, 0.5)").over(windowSpec)).distinct()

# Display the result
df_median_follower_count.show()

+-----+---------------+---+-------------------+---------+---------------------+
 ind| user_name|age| date_joined|join_year|median_follower_count|
+-----+---------------+---+-------------------+---------+---------------------+
 4315| MichellePrince| 36|2015-12-20 16:38:13| 2015| 22|
 3089| AbigailAli| 20|2015-10-24 11:23:51| 2015| 22|
 7528| AbigailAli| 20|2015-10-24 11:23:51| 2015| 22|
 5494| AnneAllen| 27|2015-12-16 15:20:05| 2015| 22|
 9875| BrendanJoseph| 26|2015-12-20 10:28:00| 2015| 22|
 8304| CharlesBerry| 25|2015-12-28 04:21:39| 2015| 22|
 9270| AmyBrown| 21|2015-11-08 16:14:16| 2015| 22|
 4| AdamAcosta| 20|2015-10-21 21:26:45| 2015| 22|
10029| AnthonyBaker| 20|2015-11-14 19:24:56| 2015| 22|
 1667| BernardArnold| 21|2015-12-12 01:39:29| 2015| 22|
 5730| RachelDavis| 36|2015-12-08 20:02:43| 2015| 22|
 2923| BrianNelson| 26|2015-11-11 03:20:57| 2015| 22|
 2418| AmandaAdams| 20|2015-10-21 08:27:36| 2015| 22|
 857| AndrewBurke| 20|2015-11-14 17:38:31| 2015| 22|
 6063| CoreyAndrews| 23|2015-11-25 13:36:22| 2015| 22|
 6145| EmilyHawkins| 30|2015-12-25 20:24:37| 2015| 22|
 8731|AndreaAlexander| 21|2015-11-10 09:27:42| 2015| 22|
 8237| AaronAbbott| 20|2015-10-23 16:08:41| 2015| 22|
 3156| AndrewBaker| 22|2015-12-21 08:06:54| 2015| 22|
 428| ClaudiaAdams| 20|2015-11-28 02:20:29| 2015| 22|
+-----+---------------+---+-------------------+---------+---------------------+
only showing top 20 rows